In [1]:
from typing import List, Optional, Union
from pydantic import BaseModel, Field

from langchain_community.chat_models.openai import ChatOpenAI

from langchain.graphs import Neo4jGraph
from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument
)


from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate
#from langchain.chat_models import AzureChatOpenAI
#from langchain_community.chat_models import AzureChatOpenAI
from dotenv import load_dotenv

from tqdm import tqdm
load_dotenv()



True

In [2]:
from langchain.graphs import Neo4jGraph
import os

#url = "neo4j+s://databases.neo4j.io"
from langchain_community.graphs import Neo4jGraph

url=os.getenv("NEO4J_URI","bolt://localhost:7687")
username=os.getenv("NEO4J_USER","neo4j")
password=os.getenv("NEO4J_PASSWORD","Pk@12345678")
#url=os.getenv("NEO4J_URI","neo4j+s://5292e0f3.databases.neo4j.io")
#username=os.getenv("NEO4J_USER","neo4j")
#password=os.getenv("NEO4J_PASSWORD","uT0_PcdAy6VlQSfXROAFOi34yVO5h_0aoO3Sm9w0TPM")

graph = Neo4jGraph(url=url, username=username, password=password)
print(graph)

In [3]:


class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

In [4]:
# Define mapping functions
def map_to_base_node(node: Node) -> BaseNode:
    return BaseNode(
        id=node.id,
        type=node.type,
        properties={prop.key: prop.value for prop in node.properties} if node.properties else {}
    )

def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    return BaseRelationship(
        source=BaseNode(
            id=rel.source.id,
            type=rel.source.type,
            properties={prop.key: prop.value for prop in rel.source.properties} if rel.source.properties else {}
        ),
        target=BaseNode(
            id=rel.target.id,
            type=rel.target.type,
            properties={prop.key: prop.value for prop in rel.target.properties} if rel.target.properties else {}
        ),
        type=rel.type,
        properties={prop.key: prop.value for prop in rel.properties} if rel.properties else {}
    )

In [5]:
class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

llm=AzureChatOpenAI(azure_deployment=os.environ['EmbeddingModelDeploymentName'],model="gpt-35-turbo-16k",azure_endpoint=os.environ['OpenAIEndpoint'],api_key=os.environ['OpenAIKey'],api_version=os.environ['OpenAIVersion'])

print(llm)

In [6]:
os.environ["OPENAI_API_KEY"] = 'fe77aff6afd640f78539d8fcad008a4c'
# OpenAI API configuration
llm=AzureChatOpenAI(azure_deployment=os.environ['EmbeddingModelDeploymentName'],model="gpt-35-turbo-16k",azure_endpoint=os.environ['OpenAIEndpoint'],api_key=os.environ['OpenAIKey'],api_version=os.environ['OpenAIVersion'])

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
    [(
      "system",
      f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"), 
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.  
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial. 
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination."""),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [7]:
from langchain_core.documents import Document
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.run(document.page_content)
    print(data.nodes)
    print(data.rels)
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    graph.add_graph_documents([graph_document])

In [8]:
print(graph)

In [13]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter

# Read the wikipedia article
#raw_documents = WikipediaLoader(query="Walt Disney").load()
raw_documents = WikipediaLoader(query="Virat Kohli").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=24)

# Only take the first the raw_documents
documents1 = text_splitter.split_documents(raw_documents[:])
print(len(documents))

1


In [15]:
print(len(documents1))
documents =documents1 #text_splitter.split_documents(raw_documents[:1])
print(len(documents))

25
25


In [16]:


for i, d in tqdm(enumerate(documents), total=len(documents)):
    extract_and_store_graph(d)

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

[Node(id='Virat Kohli', type='person', properties=[Property(key='name', value='Virat Kohli'), Property(key='birthDate', value='5 November 1988')]), Node(id='Indian national cricket team', type='team', properties=[Property(key='name', value='Indian national cricket team')]), Node(id='Royal Challengers Bengaluru', type='team', properties=[Property(key='name', value='Royal Challengers Bengaluru')]), Node(id='Delhi', type='team', properties=[Property(key='name', value='Delhi')]), Node(id='IPL', type='league', properties=[Property(key='name', value='IPL'), Property(key='team', value='Royal Challengers Bengaluru')]), Node(id='T20I', type='format', properties=[Property(key='name', value='T20I')]), Node(id='ODI', type='format', properties=[Property(key='name', value='ODI')]), Node(id='ICC Test mace', type='trophy', properties=[Property(key='name', value='ICC Test mace')]), Node(id='2011 Cricket World Cup', type='tournament', properties=[Property(key='name', value='2011 Cricket World Cup'), Pro

  8%|██████▋                                                                            | 2/25 [00:42<07:07, 18.60s/it]

[Node(id='Virat Kohli', type='person', properties=[Property(key='seniorCareer', value='List A cricket, playing against Services in the Ranji One-Day Trophy'), Property(key='internationalCareer', value='representing India since included in the ODl squad for the tour of Sri Lanka'), Property(key='achievements', value='won 2011 Cricket World Cup, won 2013 ICC Champions Trophy, won several Asia Cups'), Property(key='youthCareer', value='started in October 2002 at Luhnu cricket ground, scored 15 runs in debut match, first half-century against Haryana'), Property(key='domesticCareer', value='played for Delhi, highest run-scorer in 2002 season with 172 runs, captain of Under-15 team in 2003-04 season'), Property(key='firstClassDebut', value='made debut on 23 November 2006 against Tamil Nadu in Ranji Trophy'), Property(key='emotionalInnings', value="scored 90 runs despite father's passing away during the match"), Property(key='leagueCricket', value='played for Royal Challengers Bengaluru since

 12%|█████████▉                                                                         | 3/25 [01:47<14:38, 39.95s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 24/25 [12:04<00:29, 29.43s/it]

[Node(id='ICC World Test Championship', type='tournament', properties=[]), Node(id='2021–2023 ICC World Test Championship', type='tournament', properties=[]), Node(id='final', type='match', properties=[Property(key='winner', value='Australia'), Property(key='marginOfVictory', value='209 runs')]), Node(id='Australia', type='team', properties=[Property(key='tournamentWins', value='1')]), Node(id='India', type='team', properties=[Property(key='tournamentWins', value='0')])]
[Relationship(source=Node(id='2021–2023 ICC World Test Championship', type='tournament'), target=Node(id='final', type='match'), type='hasFinal'), Relationship(source=Node(id='final', type='match'), target=Node(id='Australia', type='team'), type='winner'), Relationship(source=Node(id='final', type='match'), target=Node(id='India', type='team'), type='loser'), Relationship(source=Node(id='Australia', type='team'), target=Node(id='ICC World Test Championship', type='tournament'), type='participant'), Relationship(source=

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [12:18<00:00, 24.71s/it]

[Node(id='Indian Premier League', type='event', properties=[Property(key='name', value='Indian Premier League'), Property(key='season', value='14'), Property(key='year', value='2021'), Property(key='sponsor', value='VIVO')]), Node(id='Mumbai Indians', type='team', properties=[Property(key='name', value='Mumbai Indians'), Property(key='defendingChampions', value='true')]), Node(id='Punjab Kings', type='team', properties=[Property(key='name', value='Punjab Kings'), Property(key='previousName', value='Kings XI Punjab')]), Node(id='Board of Control for Cricket in India', type='organization', properties=[Property(key='name', value='Board of Control for Cricket in India'), Property(key='established', value='2007')]), Node(id='Chennai Super Kings', type='team', properties=[Property(key='name', value='Chennai Super Kings')]), Node(id='Kolkata Knight Riders', type='team', properties=[Property(key='name', value='Kolkata Knight Riders')]), Node(id='Ruturaj Gaikwad', type='person', properties=[Pro

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [12:18<00:00, 29.53s/it]


# Specify which node labels should be extracted by the LLM
allowed_nodes = ["Person", "Company", "Location", "Event", "Movie", "Service", "Award"]

for i, d in tqdm(enumerate(documents), total=len(documents)):
    extract_and_store_graph(d, allowed_nodes)

In [48]:
#Query the knowledge graph in a RAG application
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=llm,
    qa_llm=llm,
    validate_cypher=True, # Validate relationship directions
    verbose=True
)
#cypher_chain.run("When The Walt Disney Company ?")
#cypher_chain.run("Who is Virat Kohli ?")
#cypher_chain.run("from which country Virat Kohli Played Cricket ?")
#cypher_chain.run("What is the age of Virat Kohli?")
#cypher_chain.run("What are the awards Virat Kohli received and on which year?")
cypher_chain.run("which domestic league Virat Kohli plays for ?")
cypher_chain.run("Where ICC T20 world cup held ?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:person {name: "Virat Kohli"})-[:PLAYS_FOR]->(t:team)
RETURN t.name
Full Context:
[{'t.name': 'Indian national cricket team'}, {'t.name': 'Royal Challengers Bengaluru'}, {'t.name': 'Delhi'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information about where the ICC T20 World Cup was held."